In [1]:
import pandas as pd
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance

In [3]:
def movieLensDataLoad(type):
    ## user 영화 별점 data 
    return pd.read_csv("/Users/yoonjae/Desktop/DataScience/Week6/"+type+"/ratings.csv")

ratings = movieLensDataLoad('ml-20m')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [5]:
## User x Movie Matrix
UM_matrix_ds = ratings.pivot(index='userId', columns='movieId', values='rating')

In [6]:
UM_matrix_ds

movieId,1,2,3,4,5,6,7,8,9,10,...,131241,131243,131248,131250,131252,131254,131256,131258,131260,131262
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4.0,NaN,5.0,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def distance_cosine(a,b):
    return 1-distance.cosine(a,b)

def disance_corr(a,b):
    return 1-distance.correlation (a,b)

def distance_euclidean(a,b):
    return 1/(distance.euclidean(a,b)+1)

In [9]:
def nearest_neighbor_user(user, topN, simFunc):

    base_user = UM_matrix_ds.loc[user].dropna()
    ratedIndex = base_user.index
    nn = {}
    
    ## Brote Force Compute
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        
        if uid == user:
            continue
            
        for i in ratedIndex:
            if math.isnan(row[i]) == False:
                interSectionU1.append(base_user[i])
                interSectionU2.append(row[i])
       
        ## At least 3 intersection items
        if len(interSectionU1) < 3 :
            continue
            
        ## similarity functon
        sim = simFunc(interSectionU1,interSectionU2)
        
        if math.isnan(sim) == False:
            nn[uid] = sim
            
    ## top N returned         
    # return sorted(nn.items(),key=itemgetter(1))[:-(topN+1):-1]
    return sorted(nn.items(),key=itemgetter(1))[:topN]

In [40]:
def nearest_neighbor_user(user, topN, simFunc):

    base_user = UM_matrix_ds.loc[user].dropna()
    ratedIndex = base_user.index
    nn = {}
        
    ## Brote Force Compute
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        
        if uid == user:
            continue
            
        for i in ratedIndex:
            if math.isnan(row[i]) == False:
                interSectionU1.append(base_user[i])
                interSectionU2.append(row[i])
       
        ## At least 3 intersection items
        if len(interSectionU1) < 3 :
            continue
            
        ## similarity functon
        sim = simFunc(interSectionU1,interSectionU2)
        
        if math.isnan(sim) == False:
            nn[uid] = sim

    ## top N returned         
    return sorted(nn.items(),key=itemgetter(1))[:-(topN+1):-1]
    # return sorted(nn.items(),key=itemgetter(1))[:topN]

In [44]:
test_set = pd.read_csv("/Users/yoonjae/Desktop/DataScience/Week6/ml-20m/test.csv")

test_set

,userId,movieId,rating,timestamp
0,11,500,4.5,1230858949
1,18,4428,3.5,1262461295
2,24,4308,2.0,1015727461
3,35,60,4.0,1164498353
4,50,924,5.0,1182349293
5,54,3194,4.0,975440547
6,58,32587,4.5,1144061539
7,70,2890,3.0,1020294249
8,83,296,5.0,1112724196
9,91,2863,4.5,1111558557


In [91]:
st = time.time()
for index, row in test_set.iterrows():
    print(row['userId'], '번 유저와 가장 가까운 이웃 3명 :')
    print(nearest_neighbor_user(row['userId'], 3, distance.euclidean))
print(time.time()-st, 'sec')

11.0 번 유저와 가장 가까운 이웃 3명 :
[(562, 0.0), (992, 0.0), (1801, 0.0)]
18.0 번 유저와 가장 가까운 이웃 3명 :
[(111, 0.0), (714, 0.0), (1543, 0.0)]
24.0 번 유저와 가장 가까운 이웃 3명 :
[(7918, 0.0), (8429, 0.0), (12132, 0.0)]
35.0 번 유저와 가장 가까운 이웃 3명 :
[(2463, 0.0), (2588, 0.0), (3866, 0.0)]
50.0 번 유저와 가장 가까운 이웃 3명 :
[(921, 0.0), (961, 0.0), (1097, 0.0)]
54.0 번 유저와 가장 가까운 이웃 3명 :
[(3156, 0.0), (15726, 0.0), (16195, 0.0)]
58.0 번 유저와 가장 가까운 이웃 3명 :
[(3362, 0.0), (16368, 0.0), (24094, 0.0)]
70.0 번 유저와 가장 가까운 이웃 3명 :
[(1978, 0.0), (2831, 0.0), (3244, 0.0)]
83.0 번 유저와 가장 가까운 이웃 3명 :
[(131238, 0.0), (176, 0.0), (131559, 0.0)]
91.0 번 유저와 가장 가까운 이웃 3명 :
[(29123, 0.0), (59007, 0.0), (71894, 0.0)]
7568.720891952515 sec


In [32]:
def predictRating_specific(userid, mid, nn = 20000, simFunc = distance_euclidean) :
   
    neighbor = nearest_neighbor_user(userid, nn, simFunc)
    neighbor_id = [id for id,sim in neighbor]
    
    ## neighboorhood's movie : al least 4 ratings
    neighbor_movie = UM_matrix_ds.loc[neighbor_id]\
                    .dropna(1, how='all', thresh = 4 )
    neighbor_dic = (dict(neighbor))
    ret = [] # ['movieId', 'predictedRate']
    
    ## rating predict by my similarities 
    for movieId, row in neighbor_movie.iteritems():
        if movieId == mid:
            jsum, wsum = 0, 0
            for v in row.dropna().iteritems():
                sim = neighbor_dic.get(v[0],0)
                jsum += sim
                wsum += (v[1]*sim)
            return [movieId, wsum/jsum]

In [33]:
predicted = []

In [34]:
for index, row in test_set.iterrows():
    uid = row['userId']
    mid = row['movieId']
    predicted.append([uid, predictRating_specific(uid, mid)])
predicted

[[11.0, [500, 4.0247442533446547]],
 [18.0, [4428, 3.7792301932228476]],
 [24.0, [4308, 3.9928693316359096]],
 [35.0, [60, 3.333819825722129]],
 [50.0, [924, 4.1483826202183787]],
 [54.0, [3194, 3.6828067374423297]],
 [58.0, [32587, 4.1874632064947086]],
 [70.0, [2890, 3.4867384741059571]],
 [83.0, [296, 4.1943253644795879]],
 [91.0, [2863, 3.8389417383723923]]]

In [35]:
def RMSE(X, left_col, right_col):
    return(np.sqrt(np.mean( (X[left_col] - X[right_col])**2 )))

def MAE(X, left_col, right_col):
    return(np.mean(np.absolute(X[left_col] - X[right_col])) )

In [36]:
df = pd.DataFrame(test_set)

In [37]:
euclidean = []
for row in predicted:
    euclidean.append(row[1][1])
df['euclidean'] = pd.Series(euclidean)
df

,userId,movieId,rating,timestamp,euclidean
0,11,500,4.5,1230858949,4.024744
1,18,4428,3.5,1262461295,3.779230
2,24,4308,2.0,1015727461,3.992869
3,35,60,4.0,1164498353,3.333820
4,50,924,5.0,1182349293,4.148383
5,54,3194,4.0,975440547,3.682807
6,58,32587,4.5,1144061539,4.187463
7,70,2890,3.0,1020294249,3.486738
8,83,296,5.0,1112724196,4.194325
9,91,2863,4.5,1111558557,3.838942


In [38]:
MAE(df, 'rating', 'euclidean')

0.6848354252890534

In [39]:
RMSE(df, 'rating', 'euclidean')

0.83460941065427552